# Module 4.2. - EDA


### Fabric Prerequistis

You need to have Lakehouse enabled and connected. 

Link to Lakehouse (replace these strings)
- Tables: `abfss://Fabric_2024@onelake.dfs.fabric.microsoft.com/LK_flights.Lakehouse/Tables`
- Files: `abfss://Fabric_2024@onelake.dfs.fabric.microsoft.com/LK_flights.Lakehouse/Files`

You will also need:
- PySpark notebook and connect it to the Fabric standard session

Data:
- Have delta tables created with flights data

In this module, you will analyze flight data, perform simple Exploratory Data Analysis (EDA), and build a machine learning model to predict whether an upcoming flight will be delayed. You will store and retrieve data from a Lakehouse, build a prediction model using PySpark, and run experiments to evaluate the model's performance.

We'll go through these steps:

1) Perform Simple EDA using PySpark notebooks.
2) Explore and understand the flight data stored in the Lakehouse.
3) Prepare and build a machine learning model that predicts flight delays.
3) Train the model and run experiments.
4) Switch between PySpark, SQL, and R to show how different languages can be used in the notebook.

## Step 1: Load Flight Data from the Lakehouse

In [ ]:
df_csv = spark.read.format("csv").option("header","true").load("Files/flights.csv")

#####
# or
#####
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CSVtoDelta").getOrCreate()

csv_file_path = "Files/flights.csv"


df_csv = spark.read.format("csv") \
    .option("header", "true") \  
    .option("inferSchema", "true") \  
    .load(csv_file_path)


df_csv.show(5)


And add it to the delta table

In [ ]:
# Step 1: Define the path in the Lakehouse where the Delta table will be saved
delta_table_path = "Tables/flights_table"

# Step 7: Save the DataFrame as a Delta table in the Lakehouse
df_csv.write.format("delta").mode("overwrite").save(delta_table_path)

In [ ]:
# Step 1: Initialize Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FlightDelayEDA").getOrCreate()

# Step 2: Define the Lakehouse path to the Delta Table
lakehouse_table_path = "abfss://Fabric_2024@onelake.dfs.fabric.microsoft.com/LK_flights.Lakehouse/Tables/nyctaxi_prep"

# Step 3: Read the flight data from Delta Table in Lakehouse
df_flight = spark.read.format("delta").load(lakehouse_table_path)

# Step 4: Display the first few rows of the flight data
df_flight.show(5)


Alternatively, you can use SQL within the notebook:

In [ ]:
%%sql

SELECT * FROM delta.`abfss://Fabric_2024@onelake.dfs.fabric.microsoft.com/LK_flights.Lakehouse/Tables/nyctaxi_prep` LIMIT 5;


Or use R for the same task:

In [ ]:
%%sparkr

library(SparkR)

df_flight <- read.df("abfss://Fabric_2024@onelake.dfs.fabric.microsoft.com/LK_flights.Lakehouse/Tables/nyctaxi_prep", source = "delta")
head(df_flight)


## Step 2: Simple Exploratory Data Analysis (EDA)

Check the Schema:

In [ ]:
# Display the schema of the flight data
df_flight.printSchema()


Summary Statistics:

In [ ]:
# Display summary statistics for all numerical columns
df_flight.describe().show()


Check for Missing Data:

In [ ]:
from pyspark.sql.functions import col, count, when

# Check for missing data by counting null values in each column
missing_data = df_flight.select([count(when(col(c).isNull(), c)).alias(c) for c in df_flight.columns])
missing_data.show()


Distribution of Flight Delays:

In [ ]:
# Group by the 'is_delay' column to check the distribution of delayed vs non-delayed flights
df_flight.groupBy("is_delay").count().show()


And we can  explore data with SQL for aggregation:

In [ ]:
%%sql
-- Count the number of delayed and non-delayed flights

SELECT is_delay, COUNT(*) AS count 
FROM delta.`abfss://<your-container>@<your-storage-account>.dfs.core.windows.net/delta/flight_data`
GROUP BY is_delay;


## Step 3: Data Preparation for Machine Learning

We will prepare the flight data for building the machine learning model. We'll transform the features and prepare the dataset for training.

### Feature Selection:

We'll use key features such as departure time, origin, destination, and carrier to predict the delay.

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

# Index categorical columns (e.g., carrier, origin, destination)
indexer_carrier = StringIndexer(inputCol="carrier", outputCol="carrier_index")
indexer_origin = StringIndexer(inputCol="origin", outputCol="origin_index")
indexer_dest = StringIndexer(inputCol="destination", outputCol="dest_index")

df_flight = indexer_carrier.fit(df_flight).transform(df_flight)
df_flight = indexer_origin.fit(df_flight).transform(df_flight)
df_flight = indexer_dest.fit(df_flight).transform(df_flight)

# Combine all relevant features into a single vector column
assembler = VectorAssembler(inputCols=["carrier_index", "origin_index", "dest_index", "departure_time"], outputCol="features")
df_flight = assembler.transform(df_flight)


## Step 4: Build and Train a Machine Learning Model

Build a Logistic Regression model to predict whether a flight will be delayed or not.

1) Train-Test Split:

In [ ]:
# Split the data into training (80%) and test (20%) sets
train_df, test_df = df_flight.randomSplit([0.8, 0.2], seed=42)


2) Logistic Regression Model:

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Initialize the Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="is_delay")

# Train the model
lr_model = lr.fit(train_df)

# Display the model summary
lr_model.summary


## Step 5: Run Experiments and Evaluate the Model

We will evaluate the performance of the trained model using metrics such as accuracy and Area Under the ROC Curve (AUC).

1) Make Predictions:

In [ ]:
# Use the model to make predictions on the test data
predictions = lr_model.transform(test_df)

# Show the predictions
predictions.select("features", "is_delay", "prediction", "probability").show(5)


2) Evaluate the Model:

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Initialize the evaluator for AUC metric
evaluator = BinaryClassificationEvaluator(labelCol="is_delay", metricName="areaUnderROC")

# Calculate AUC for the test dataset
auc = evaluator.evaluate(predictions)
print(f"AUC: {auc}")


Alternatively, you can calculate accuracy using SQL:

In [ ]:
%%sql

-- Calculate accuracy of the model
SELECT COUNT(*) * 1.0 / SUM(CASE WHEN prediction = is_delay THEN 1 ELSE 0 END) AS accuracy
FROM predictions;


## Step 6: Track Experiments and Run Multiple Models

We will Track the model’s performance across different experiments and retrain if necessary.

Using MLFlow in Microsoft Fabric allows you to track and compare multiple model versions easily.

In [ ]:
import mlflow
import mlflow.spark

# Start MLflow experiment tracking
mlflow.start_run()

# Log parameters, metrics, and the model
mlflow.log_param("model", "LogisticRegression")
mlflow.log_metric("AUC", auc)
mlflow.spark.log_model(lr_model, "flight_delay_model")

# End MLflow run
mlflow.end_run()


## Step 7: Making Predictions for the Next Flight (Inference)

Perform inference for upcoming flights to predict whether they will be delayed.

In [ ]:
# Assuming new_flight_df contains data for an upcoming flight
new_flight_df = spark.createDataFrame([
    ("AA", "LAX", "JFK", "16:00")
], ["carrier", "origin", "destination", "departure_time"])

# Prepare the new data (index and vectorize)
new_flight_df = indexer_carrier.fit(new_flight_df).transform(new_flight_df)
new_flight_df = indexer_origin.fit(new_flight_df).transform(new_flight_df)
new_flight_df = indexer_dest.fit(new_flight_df).transform(new_flight_df)

new_flight_df = assembler.transform(new_flight_df)

# Make the prediction
new_flight_prediction = lr_model.transform(new_flight_df)

# Show the prediction
new_flight_prediction.select("features", "prediction", "probability").show()


## Switching to SQL or R

We can  also make predictions using SQL if the model has been registered in MLFlow:

In [ ]:
%%sql
SELECT *, PREDICT(flight_delay_model, carrier_index, origin_index, dest_index, departure_time)
FROM delta.`abfss://<your-container>@<your-storage-account>.dfs.core.windows.net/delta/new_flight_data`;


Or use R for Training the Model:

In [ ]:
library(SparkR)

# Split the dataset into training and testing sets
splits <- randomSplit(df_flight, c(0.8, 0.2), seed = 42)
train_df <- splits[[1]]
test_df <- splits[[2]]

# Train a Logistic Regression model in R
lr_model <- spark.logit(train_df, label = "is_delay", features = c("carrier_index", "origin_index", "dest_index", "departure_time"))

# Make predictions and evaluate the model
predictions <- predict(lr_model, test_df)
head(predictions)
